In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from google.colab import drive
from IPython.display import Image
import json
import re
import pandas as pd

In [ ]:
import cv2
!pip install mediapipe
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.7)



In [ ]:
drive.flush_and_unmount()
drive.mount('/content/gdrive')
#drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# check number of files in each directory

import glob

classnames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
#lassnames = ['W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
totalcount = 0

for name in classnames:
  data_files = glob.glob("/content/gdrive/My Drive/Capstone/Dataset/demo_test/" + name + "/*")
  count = 0
  path = "/content/gdrive/My Drive/Capstone/Dataset/handpose_test/{0}/".format(name)
  ! mkdir -pv "$path"
  for i, data_file in enumerate(data_files):
    count += 1
    totalcount += 1
    file_name = data_file.split('/')[-1].replace('.png', '')

    image = cv2.flip(cv2.imread(data_file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


    with open("/content/gdrive/My Drive/Capstone/Dataset/handpose_test/{0}/{1}.json".format(name, file_name), 'w') as f:

       if not results.multi_hand_landmarks:
          continue
       for hand_landmarks in results.multi_hand_landmarks:
          oneline = str.join(" ", str(hand_landmarks).splitlines())
          
          """
          Ugly text pre-processing
          """
          for r in (('x:', '"x":'), ('z:', ',"z":'), ('visibility:', ',"visibility":'), ('y:', ',"y":'),('presence:', ',"presence":')):
              oneline = oneline.replace(*r)
          landmarks = re.findall(r'\{(.*?)\}', oneline)
          json_o = '{'
          for idx, x in enumerate(landmarks):
            json_o += '"{0}": '.format(idx) + '{' + x + '}, \n'
          json_o = json_o[:-3] + '}'

          #Finally gets json in right format! Yay
          hello = json.loads(json_o)
          f.write(str(hello))

  print(name + ":" + str(count))
print("total number of images:" + str(totalcount))

In [ ]:
help(mp.solutions.hands)

In [ ]:
classnames = ['A']
totalcount = 0

for name in classnames:
  data_files = glob.glob("/content/gdrive/My Drive/Capstone/Dataset/demo_test/" + name + "/*")
  count = 0
  path = "/content/gdrive/My Drive/Capstone/Dataset/handpose_test/{0}/".format(name)
  ! mkdir -pv "$path"
  for i, data_file in enumerate(data_files[:1]):
    count += 1
    totalcount += 1
    file_name = data_file.split('/')[-1].replace('.png', '')

    image = cv2.flip(cv2.imread(data_file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.multi_hand_landmarks:
      continue
    for hand_landmarks in results.multi_hand_landmarks:
      oneline = str.join(" ", str(hand_landmarks).splitlines())
      
      """
      Ugly text pre-processing
      """
      for r in (('x:', '"x":'), ('z:', ',"z":'), ('visibility:', ',"visibility":'), ('y:', ',"y":'),('presence:', ',"presence":')):
          oneline = oneline.replace(*r)
      landmarks = re.findall(r'\{(.*?)\}', oneline)
      json_o = '{'
      for idx, x in enumerate(landmarks):
        json_o += '"{0}": '.format(idx) + '{' + x + '}, \n'
      json_o = json_o[:-3] + '}'

      #Finally gets json in right format! Yay
      hello = json.loads(json_o)
      image = []
      for idx, i in enumerate(hello.items()):
          image.append([i[1]['x'], i[1]['y'], i[1]['z']])
      
      print(tf.convert_to_tensor(image))
  print(name + ":" + str(count))
print("total number of images:" + str(totalcount))

In [ ]:
df = pd.DataFrame(columns=['x', 'class'])

import glob
classnames = ['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
#classnames = ['W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
totalcount = 0

for name in classnames:
  data_files = glob.glob("/content/gdrive/My Drive/Capstone/Dataset/handpose_test/"+ name + "/*")
  count = 0
  for i, data_file in enumerate(data_files):
      with open(data_file, 'r') as json_file: 
          data = json_file.read().replace('\n', '')
          data = data.replace("'", '"')
          try:
            data = json.loads(data) 
          except:
            continue
      image = []
      for idx, i in enumerate(data.items()):
          image.append([i[1]['x'], i[1]['y'], i[1]['z']])
      image = np.array(image)
      df.loc[len(df)] = [image, name]
      

  print(name + ":" + str(count))
print("total number of images:" + str(totalcount))
df.to_csv("/content/gdrive/My Drive/Capstone/Dataset/handpose_test.csv")


In [ ]:
#df = df[(df['class'] < 26)]
df.to_csv("/content/gdrive/My Drive/Capstone/Dataset/df.csv")

In [ ]:
def converter(x):
    astr = x.split('\n')
    astr = [ x.replace('[','').replace(']','').split('\n') for x in astr]
    astr = [ x[0].split() for x in astr]
    new_arr = np.array( astr, 'float32' )
    return new_arr

df = pd.read_csv("/content/gdrive/My Drive/Capstone/Dataset/df.csv", usecols=['x', 'class'])
#print(df.loc[0, 'x'])
df2 = pd.DataFrame(columns=['x', 'class'])
for idx, row in df.iterrows():
  arr = converter(row['x'])
  df2.loc[len(df2)] = [arr, row['class']]


In [ ]:
df = df2.copy()

In [ ]:
from collections import Counter

Counter(df['class'].values)

In [ ]:
#df = pd.read_csv("/content/gdrive/My Drive/Capstone/Dataset/handpose_test.csv", usecols=['x', 'class'])

#for idx, row in df.iterrows():
#  df.loc[idx, 'x'] = converter(row['x'])

df['class'] = pd.Categorical(df['class'])
df['class'] = df['class'].cat.codes
#test_df = df.copy()


In [ ]:
df

In [ ]:
ds
for feat, targ in ds.take(5):
  #print ('Features: {}, Target: {}'.format(feat, targ))
  print(len(feat), len(targ))

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('class')
  vals = [tf.convert_to_tensor(x) for x in dataframe['x'].values]
  ds = tf.data.Dataset.from_tensor_slices((vals, labels))
  print(ds)
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds



In [ ]:
ds = df_to_dataset(df)
for feat, targ in ds.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

In [ ]:
test_df = test_df[(test_df['class'] < 26)]


In [ ]:
test_ds = df_to_dataset(test_df)
for feat, targ in test_ds.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

In [ ]:
test_ds.element_spec

In [ ]:

model = Sequential([
    Dense(32, input_shape=(21, 3), kernel_initializer='he_uniform'),
    Flatten(),
    Activation('relu'),
    Dense(26),
    Activation('sigmoid'),
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(ds, epochs=50)

In [ ]:
model.save("/content/gdrive/My Drive/Capstone/models/handpose.h5")

In [ ]:
model = keras.models.load_model("/content/gdrive/My Drive/Capstone/models/handpose.h5")

In [ ]:
test_ds

In [ ]:
print("Evaluate")
result = model.evaluate(test_ds)
dict(zip(model.metrics_names, result))

In [ ]:
model.predict(test_ds[:1])

In [ ]:
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5)

image = cv2.flip(cv2.imread("/content/gdrive/My Drive/Capstone/Dataset/demo_training_set/F/MaxF299.png"), 1)
# Convert the BGR image to RGB before processing.
results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
annotated_image = image.copy()

for hand_landmarks in results.multi_hand_landmarks:

    mp_drawing.draw_landmarks(
        annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imwrite(
        'annotated_image.png', cv2.flip(image, 1))
    oneline = str.join(" ", str(hand_landmarks).splitlines())

    """
    Ugly text pre-processing
    """
    for r in (('x:', '"x":'), ('z:', ',"z":'), ('visibility:', ',"visibility":'), ('y:', ',"y":'),('presence:', ',"presence":')):
        oneline = oneline.replace(*r)
    landmarks = re.findall(r'\{(.*?)\}', oneline)
    json_o = '{'
    for idx, x in enumerate(landmarks):
        json_o += '"{0}": '.format(idx) + '{' + x + '}, \n'
    json_o = json_o[:-3] + '}'

    #Finally gets json in right format! Yay
    hello = json.loads(json_o)

    image_arr = []
    for idx, i in enumerate(hello.items()):
        #print(i)
        image_arr.append([i[1]['x'], i[1]['y'], i[1]['z']])
    
    
    
    #print(image_arr)
    image_arr = np.array(image_arr)
    image_arr = np.expand_dims(image_arr, 0)
    im_tensor = tf.convert_to_tensor(image_arr)

    pred = model.predict(im_tensor, batch_size=1)
    print(np.argmax(pred))